In this notebook we compute approximate solutions of the reduced Ostrovsky equation

\begin{align}
    u_t + uu_x & = v \\
    v_x = u.
\end{align}

There is an implicit constraint that $\int u dx = \int v dx = 0$.  We use a pseudospectral method in space and compute $v$ by taking the Fourer transform of $u$, dividing by $i\xi$ (where $\xi$ is the wavenumber), and taking the inverse Fourier transform.  One problem with this is dealing with $\xi=0$.  Due to the constraint, we know that the $\xi=0$ component of the solution must vanish, so we just replace the zero value of $\xi$ with an arbitrary number (1) when dividing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
from ipywidgets import interact, FloatSlider
import scipy.integrate
from scipy.special import erf
font = {'size'   : 15}
matplotlib.rc('font', **font)

In [ ]:
def rk3(u,xi,rhs):
    y2 = u + dt*rhs(u,xi)
    y3 = 0.75*u + 0.25*(y2 + dt*rhs(y2,xi))
    u_new = 1./3 * u + 2./3 * (y3 + dt*rhs(y3,xi))
    return u_new

In [ ]:
scenario = 'shock'

# Grid
m = 1024
L = 2*np.pi
x = np.arange(-m/2,m/2)*(L/m)
xi = np.fft.fftfreq(m)*m*2*np.pi/L
# modified wavenumber vector for computing v as integral of u:
xi2 = np.copy(xi); xi2[0] = 1.  


def rhs(u, xi, xi2=xi2):
    uhat = np.fft.fft(u)
    vhat = uhat/(1j*xi2)
    v = np.real(np.fft.ifft(vhat))
    assert(np.sum(v)<1.e-13)
    return -u*np.real(np.fft.ifft(1j*xi*uhat)) + v


dt = 1.73/(m/2) / 2

if scenario == 'no shocks':
    u = -np.sin(x)/4; tmax = 20
elif scenario == 'shock':
    u = -np.sin(x)/2; tmax = 3

assert(np.sum(u)<1.e-13)

uhat2 = np.abs(np.fft.fft(u))

num_plots = 50
nplt = np.floor((tmax/num_plots)/dt)
nmax = int(round(tmax/dt))

fig = plt.figure(figsize=(12,8))
axes = fig.add_subplot(111)
line, = axes.plot(x,u,lw=3)
axes.set_xlabel(r'$x$',fontsize=30)
axes.set_ylabel(r'$u$',fontsize=30)
plt.tight_layout()
plt.close()

frames = [u.copy()]
tt = [0]
uuhat = [uhat2]

for n in range(1,nmax+1):
    u_new = rk3(u,xi,rhs)
    u = u_new.copy()
    t = n*dt
    # Plotting
    if np.mod(n,nplt) == 0:
        frames.append(u.copy())
        tt.append(t)
        uhat2 = np.abs(np.fft.fft(u))
        uuhat.append(uhat2)
        
def plot_frame(i):
    line.set_data(x,frames[i])
    power_spectrum = np.abs(uuhat[i])**2
    axes.set_title('t= %.2e' % tt[i])
    axes.set_xlim((-np.pi,np.pi))
    axes.set_ylim((-1,2))
    
anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                   frames=len(frames), interval=100)

HTML(anim.to_jshtml())

# Characteristics

In [ ]:
scenario = 'Gaussian'
case = 3

# Grid
m = 1024
L = 2*np.pi
x = np.arange(-m/2,m/2)*(L/m)
xi = np.fft.fftfreq(m)*m*2*np.pi/L
xi2 = np.copy(xi)
xi2[0] = 1.


def rhs(u, xi, xi2=xi2):
    uhat = np.fft.fft(u)
    vhat = uhat/(1j*xi2)
    v = np.real(np.fft.ifft(vhat))
    assert(np.sum(v)<1.e-13)
    return -u*np.real(np.fft.ifft(1j*xi*uhat)) + v

dt = 1.73/(m/2) / 1.

ymin = -0.6; ymax = 1.0


if scenario == 'no shocks':
    u = -np.sin(x)/4; tmax = 20
elif scenario == '1 shock':
    u = -np.sin(x)/2; tmax = 3
elif scenario == '2 shocks':
    u = -np.sin(2*x)/11.5; tmax = 50
elif scenario == '4 shocks':
    u = -np.sin(4*x)/(3*4**2-6.); tmax = 50
elif scenario == 'Gaussian':
    if case == 1:
        b = -0.0135; l=0.3; tmax = 100
    elif case == 2:
        b = -0.0165; l=0.3; tmax = 100
    elif case == 3:
        b = 0.05; l=0.3; tmax = 18
        
    ymin = -0.02; ymax = 0.06
    u = b*np.exp(-x**2/l**2) - b*l/(2*np.sqrt(np.pi)) * erf(np.pi/l)
else:
    raise(Exception('invalid scenario'))

assert(np.sum(u)<1.e-13)

uhat2 = np.abs(np.fft.fft(u))

num_plots = 100
nplt = np.floor((tmax/num_plots)/dt)
nmax = int(round(tmax/dt))

fig, axes = plt.subplots(2,1,figsize=(12,8))
line, = axes[0].plot(x,u,lw=3)

# Characteristics
charplots = []
charskip = 20
x0s = x[::charskip]
ichars = np.arange(len(x0s))*charskip  # Indices of starting points of characteristics
charvals = []
for x0 in x0s:
    charplots.append(axes[1].plot([],[],'-k')[0])
    charvals.append([x0])   # recorded only at plot times
    
charcurvals = np.copy(x0s)  # updated at every time step


axes[0].set_xlabel(r'$x$',fontsize=30)
#plt.tight_layout()
plt.close()

frames = [u.copy()]
tt = [0]
uuhat = [uhat2]

for n in range(1,nmax+1):
    u_new = rk3(u,xi,rhs)
    u = u_new.copy()
    t = n*dt
    
    # Update characteristics using explicit Euler
    for i in range(len(x0s)):
        charcurvals[i] = charcurvals[i] + dt*u[ichars[i]]
    
    # Plotting
    if np.mod(n,nplt) == 0:
        frames.append(u.copy())
        tt.append(t)
        uhat2 = np.abs(np.fft.fft(u))
        uuhat.append(uhat2)
                   
        for i in range(len(charvals)):
            charvals[i].append(charcurvals[i])
        
def plot_frame(i):
    line.set_data(x,frames[i])
    power_spectrum = np.abs(uuhat[i])**2
    axes[0].set_title('t= %.2e' % tt[i])
    axes[0].set_xlim((-np.pi,np.pi))
    axes[0].set_ylim((ymin,ymax))
    
    axes[1].set_xlim(-np.pi,np.pi)
    axes[1].set_ylim(0,tmax)
                   
    for j, charline in enumerate(charplots):
        charline.set_data(charvals[j][:i],tt[:i])

In [ ]:
anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                   frames=len(frames), interval=100)
HTML(anim.to_jshtml())

# F

In [ ]:
F = np.zeros((len(frames),len(frames[0])))

for i, frame in enumerate(frames):
    uxx = np.diff(frames[i],2,prepend=[frames[i][-1]],append=[frames[i][0]])/dx**2
    F[i,:] = 1-3*uxx

plt.figure(figsize=(8,8))
plt.contour(x,tt,F,[0])
plt.xlabel('x')
plt.ylabel('t')

# Traveling waves 

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(6,6))

c = 1.0

u = np.linspace(-1, 2., 50)
v = np.linspace(-1.5, 1.5, 50)
 
V, U = np.meshgrid(v, u)
du = V/(U-c)
dv = U

axes.streamplot(V,U,dv,du,broken_streamlines=False,density=0.8)
axes.set_xlabel('v'); axes.set_ylabel('u');
axes.plot([0],[0],'ok')
plt.axis('image');

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,6))
plt.close()

def reduced_ostrovsky_phase_plane(c=1.,u0=0.5,xmax=2*np.pi):

    axes[0].cla()
    axes[1].cla()
    u = np.linspace(-1, 2., 50)
    v = np.linspace(-1.5, 1.5, 50)

    V, U = np.meshgrid(v, u)
    du = V/(U-c)
    dv = U


    stream = axes[0].streamplot(V,U,dv,du,broken_streamlines=False,density=0.8)
    axes[0].set_xlabel('v'); axes[0].set_ylabel('u');
    axes[0].plot([0],[0],'ok')
    axes[0].axis('image')
    
    def rhs(t,w):
        u, v = w
        return np.array([v/(u-c),u])

    w0 = np.array([u0,0.01])
    t_eval = np.linspace(0,xmax,1000)
    forwardsoln = scipy.integrate.solve_ivp(rhs,[0,xmax],w0,t_eval=t_eval,atol=1.e-12,rtol=1.e-12)
    u = forwardsoln.y[0,:]
    v = forwardsoln.y[1,:]
    x = forwardsoln.t
    axes[1].plot(x,u,'-r',lw=3)
    axes[0].plot(v,u,'-r',lw=3)
    axes[1].set_xlim(0,xmax)
    axes[1].set_ylim(-2,2)
    fig.canvas.draw_idle()
    plt.close()
    return fig
    
interact(reduced_ostrovsky_phase_plane,u0=FloatSlider(min=-0.1,max=2.0,step=0.01,value=0.5));